In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re
from bs4 import BeautifulSoup as bs
import psycopg2
from sqlalchemy import create_engine

In [554]:
conn = psycopg2.connect("host='**.rds.amazonaws.com' dbname='**' user='**' password ='**' port=5432")
engine = create_engine('postgresql+psycopg2://username:password@**.rds.amazonaws.com:5432/dbname')
sql_query = 'select * from stockinfo_stock'
df = pd.read_sql_query(sql_query,con=conn)
df = df[['hname','shcode']]
conn2 = engine.connect()
conn.close()

In [2]:
def return_recent(col):
    for i, date in enumerate(col):
        if date.endswith('(E)'):
            return i-1

In [3]:
# df = df.loc[1080:,:]

In [557]:
len_data = df.shape[0]

no_data = []
for i, data in df.iterrows():
    shcode = data['shcode']
    hname = data['hname']
    print(str(i+1), '/', str(len_data) , ':', shcode, hname)
    
    # 재무제표
    url = 'http://comp.wisereport.co.kr/company/ajax/cF1001.aspx?cmp_cd={shcode}&fin_typ=0&freq_typ=Y'.format(shcode=shcode)
    data = requests.get(url)
    text_data = re.compile('[\\r\\n\\t]').sub('', data.text)
    text_data = text_data.replace('<th class="bg r01c02 endLine line-bottom"colspan="8">연간</th>', '')
    text_data = text_data.replace('<th class="noData r01c02 bg line-bottom"colspan="8">연간</th>', '')
    text_data = text_data.replace('<span class=\'span-sub\'>(IFRS연결)</span>', '')
    text_data = text_data.replace('<span class=\'span-sub\'>(IFRS별도)</span>', '')
    ftable = pd.read_html(text_data)[0]
    
    if ftable.loc[0, '주요재무정보'] == '해당 데이터가 존재하지 않습니다.': 
        no_data.append(shcode)
        continue
    col = ftable.columns
    ftable = ftable.set_index('주요재무정보').iloc[:,:return_recent(col)]
    ftable = ftable.dropna(axis=1, how='all')

    # 수익성
    url = 'http://comp.wisereport.co.kr/company/cF4002.aspx?cmp_cd={shcode}&frq=0&rpt=1&finGubun=MAIN&frqTyp=0&cn='.format(shcode=shcode)
    data = requests.get(url)
    json_data= json.loads(data.text)
    column = {'DATA%s'%str(i+1):col[:7] for i,col in enumerate(json_data['YYMM'][:5])}
    profit = pd.DataFrame(json_data['DATA'])[['ACC_NM','DATA1', 'DATA2', 'DATA3', 'DATA4','DATA5']].set_index('ACC_NM')
    profit = profit.rename_axis(column,axis=1)
    profit = profit.loc[['매출총이익률','영업이익률','순이익률','EBITDA마진율','ROE','ROA'],:]
    profit = profit.dropna(axis=1, how='all')

    # 성장성
    url = 'http://comp.wisereport.co.kr/company/cF4002.aspx?cmp_cd={shcode}&frq=0&rpt=2&finGubun=MAIN&frqTyp=0&cn='.format(shcode=shcode)
    data = requests.get(url)
    json_data= json.loads(data.text)
    column = {'DATA%s'%str(i+1):col[:7] for i,col in enumerate(json_data['YYMM'][:5])}
    growth = pd.DataFrame(json_data['DATA'])[['ACC_NM','DATA1', 'DATA2', 'DATA3', 'DATA4','DATA5']].set_index('ACC_NM')
    growth = growth.rename_axis(column,axis=1)
    growth = growth.loc[['매출액증가율','영업이익증가율','순이익증가율','총자산증가율','유동자산증가율','유형자산증가율','자기자본증가율'],:]
    growth = growth.dropna(axis=1, how='all')

    # 안정성
    url = 'http://comp.wisereport.co.kr/company/cF4002.aspx?cmp_cd={shcode}&frq=0&rpt=3&finGubun=MAIN&frqTyp=0&cn='.format(shcode=shcode)
    data = requests.get(url)
    json_data= json.loads(data.text)
    column = {'DATA%s'%str(i+1):col[:7] for i,col in enumerate(json_data['YYMM'][:5])}
    stable = pd.DataFrame(json_data['DATA'])[['ACC_NM','DATA1', 'DATA2', 'DATA3', 'DATA4','DATA5']].set_index('ACC_NM')
    stable = stable.rename_axis(column,axis=1)
    stable = stable.loc[['부채비율','유동부채비율','비유동부채비율','순부채비율','유동비율','당좌비율','이자보상배율','금융비용부담률','자본유보율'],:]
    stable = stable.dropna(axis=1, how='all')

    # 활동성
    url = 'http://comp.wisereport.co.kr/company/cF4002.aspx?cmp_cd={shcode}&frq=0&rpt=4&finGubun=MAIN&frqTyp=0&cn='.format(shcode=shcode)
    data = requests.get(url)
    json_data= json.loads(data.text)
    column = {'DATA%s'%str(i+1):col[:7] for i,col in enumerate(json_data['YYMM'][:5])}
    act = pd.DataFrame(json_data['DATA'])[['ACC_NM','DATA1', 'DATA2', 'DATA3', 'DATA4','DATA5']].set_index('ACC_NM')
    act = act.rename_axis(column,axis=1)
    act = act.loc[['총자산회전율','자기자본회전율','순운전자본회전율','유형자산회전율','매출채권회전율','재고자산회전율','매입채무회전율'],:]
    act = act.dropna(axis=1, how='all')

    # Fundamentals
    url = 'http://comp.wisereport.co.kr/company/cF4002.aspx?cmp_cd={shcode}&frq=0&rpt=5&finGubun=MAIN&frqTyp=0'.format(shcode=shcode)
    data = requests.get(url)
    json_data= json.loads(data.text)
    column = {'DATA%s'%str(i+1):col[:7] for i,col in enumerate(json_data['YYMM'][:5])}
    value = pd.DataFrame(json_data['DATA'])[['ACC_NM','DATA1', 'DATA2', 'DATA3', 'DATA4','DATA5']].set_index('ACC_NM')
    value = value.rename_axis(column,axis=1)
    value = value.loc[['EPS', 'BPS', 'CPS','SPS','PER','PBR','PCR','PSR','EV/EBITDA','DPS','현금배당수익률','현금배당성향(%)'],:]
    value = value.dropna(axis=1, how='all')
    
    
    full_statement = pd.concat([ftable, profit, growth, stable, act, value], axis=0)
    full_statement.index = full_statement.index.str.replace(r'[\()%]', '')
    full_statement = full_statement.T
    full_statement['shcode'] = shcode
    full_statement['hname'] = hname
    full_statement.reset_index(inplace=True)
    full_statement.rename_axis({'index':'date'}, axis=1, inplace=True)
    full_statement.to_sql(name='comp_statement', con=engine, if_exists = 'append', index=False)

    # 기업개요
    comp_dummy = {}
    url = 'http://comp.wisereport.co.kr/company/c1010001.aspx?cmp_cd={shcode}'.format(shcode=shcode)
    data = requests.get(url)
    soup = bs(data.text, 'html.parser')
    try:
        res = soup.find('div', 'cmp_comment').find_all('li')
    except:
        continue
    li = []
    for indi in res:
        li.append(indi.text)
    li_text = '\n '.join(li)

    # 매출구성
    url = 'http://comp.wisereport.co.kr/company/c1020001.aspx?cmp_cd={shcode}'.format(shcode=shcode)
    data = requests.get(url)
    soup = bs(data.text, 'html.parser')
    res = soup.find('table', attrs={'id':'cTB203'})
    comp_table = pd.read_html(str(res))[0]
    sales_composition = comp_table.to_json(orient='values',force_ascii=False)
    
    comp_dummy = pd.DataFrame()
    comp_dummy.loc[0,'shcode'] = shcode
    comp_dummy.loc[0,'hname'] = hname
    comp_dummy.loc[0,'desc'] = li_text
    comp_dummy.loc[0,'sales_comp'] = sales_composition
    comp_dummy.to_sql(name='comp_summary', con=engine, if_exists = 'append', index=False)
   

1 / 2731 : 000020 동화약품
2 / 2731 : 000030 우리은행
3 / 2731 : 000040 KR모터스
4 / 2731 : 000050 경방
5 / 2731 : 000060 메리츠화재
6 / 2731 : 000070 삼양홀딩스
7 / 2731 : 000075 삼양홀딩스우
8 / 2731 : 000080 하이트진로
9 / 2731 : 000087 하이트진로2우B
10 / 2731 : 000100 유한양행
11 / 2731 : 000105 유한양행우
12 / 2731 : 000120 CJ대한통운
13 / 2731 : 000140 하이트진로홀딩스
14 / 2731 : 000145 하이트진로홀딩스우
15 / 2731 : 000150 두산
16 / 2731 : 000155 두산우
17 / 2731 : 000157 두산2우B
18 / 2731 : 000180 성창기업지주
19 / 2731 : 000210 대림산업
20 / 2731 : 000215 대림산업우
21 / 2731 : 000220 유유제약
22 / 2731 : 000225 유유제약1우
23 / 2731 : 000227 유유제약2우B
24 / 2731 : 000230 일동홀딩스
25 / 2731 : 000240 한국타이어월드와이드
26 / 2731 : 000270 기아차
27 / 2731 : 000300 대유플러스
28 / 2731 : 000320 노루홀딩스
29 / 2731 : 000325 노루홀딩스우
30 / 2731 : 000370 한화손해보험
31 / 2731 : 000390 삼화페인트
32 / 2731 : 000400 롯데손해보험
33 / 2731 : 000430 대원강업
34 / 2731 : 000480 조선내화
35 / 2731 : 000490 대동공업
36 / 2731 : 000500 가온전선
37 / 2731 : 000520 삼일제약
38 / 2731 : 000540 흥국화재
39 / 2731 : 000545 흥국화재우
40 / 2731 : 000547 흥국화재2우B
41 /

In [7]:
shcode = '005930'
hname = '삼성전자'

In [44]:
# 기업개요
comp_dummy = {}
url = 'http://comp.wisereport.co.kr/company/c1010001.aspx?cmp_cd={shcode}'.format(shcode=shcode)
data = requests.get(url)
soup = bs(data.text, 'html.parser')
try:
    res = soup.find('div', 'cmp_comment').find_all('li')
except:
    pass
li = []
for indi in res:
    li.append(indi.text)
li_text = '\n '.join(li)

# 매출구성
url = 'http://comp.wisereport.co.kr/company/c1020001.aspx?cmp_cd={shcode}'.format(shcode=shcode)
data = requests.get(url)
soup = bs(data.text, 'html.parser')
res = soup.find('table', attrs={'id':'cTB203'})
comp_table = pd.read_html(str(res))[0]
comp_table.rename_axis({'제품명':'product','구성비':'composition'}, axis=1, inplace=True)
sales_composition = comp_table.set_index('product').to_json(orient='columns', force_ascii=False)

comp_dummy = pd.DataFrame()
comp_dummy.loc[0,'shcode'] = shcode
comp_dummy.loc[0,'hname'] = hname
comp_dummy.loc[0,'desc'] = li_text
comp_dummy.loc[0,'sales_comp'] = sales_composition

In [45]:
comp_dummy

,shcode,hname,desc,sales_comp
0,005930,삼성전자,"1969년 설립 된 글로벌 기업으로, 주요사업은 CE부문(TV, 냉장고 등)과 IM...","{""composition"":{""IM"":47.0,""반도체"":34.33,""CE"":16...."


In [558]:
conn2.close()

# End